In [1]:
import os
import pandas as pd

DIR_DATA = os.path.join("..", "data")
PATH_DATA_TRAIN = os.path.join(DIR_DATA, "train.csv")
PATH_DATA_TEST = os.path.join(DIR_DATA, "test.csv")

In [2]:
df_train = pd.read_csv(PATH_DATA_TRAIN)

In [21]:
from collections import defaultdict
import math
import sys
import time
from multiprocessing import Pool, Manager, Process
import multiprocessing
from markov_helper import count_ngrams, progress_monitor

MARKOV_DEF_SMOOTHING_WEIGHT = .1
MARKOV_DEF_VOCAB_SIZE = 95
MARKOV_MULTITHREAD_NUM_WORKERS = multiprocessing.cpu_count() - 2

class MarkovModel:
    order: int
    ngram_freqs: list[dict]
    totals: list[int]
    vocab_size: int
    smoothing_weight: float
    __pool = Pool(processes=MARKOV_MULTITHREAD_NUM_WORKERS)
    __manager = Manager()
    __type = None

    def __init__(self, order: int, sequences: list = [], multithread: bool = True, verbose: bool = False, 
                 vocab_size: int = MARKOV_DEF_VOCAB_SIZE, smoothing_weight: float = MARKOV_DEF_SMOOTHING_WEIGHT):
        assert(order >= 0)

        self.order = order
        self.ngram_freqs = [defaultdict(int)] * (order + 1)
        self.totals = [0] * (order + 1)
        self.vocab_size = vocab_size
        self.smoothing_weight = smoothing_weight

        if (len(sequences) > 0):
            if isinstance(sequences[0], str):
                self.__type = str
            elif isinstance(sequences[0], list):
                self.__type = list
            else:
                raise TypeError(f'sequences argument must be either a list of strings or list of lists (is list[{type(sequences[0])}]).')

            start = time.time()

            if (multithread):
                processes = [None] * (order + 1)  
                progresses = [None] * (order + 1)

                try:
                    for n in range(order + 1):
                        progresses[n] = MarkovModel.__manager.Value('i', 0)
                        process = MarkovModel.__pool.apply_async(count_ngrams, args=(sequences, n, progresses[n]))
                        processes[n] = process   

                    if verbose:
                        monitor = Process(target=progress_monitor, args=(progresses, order + 1, len(sequences)))
                        monitor.start()
                    
                    for n in range(len(processes)):
                        (total, frequencies) = processes[n].get()
                        self.totals[n] = total
                        self.ngram_freqs[n] = frequencies
                    
                except Exception as e:
                    MarkovModel.__pool.terminate()
                    MarkovModel.__pool = Pool(processes=MARKOV_MULTITHREAD_NUM_WORKERS)
                    raise e
                finally:
                    if verbose:
                        monitor.terminate()
                    
            else:
                i = 0
                for seq in sequences:
                    i += 1
                    if i % 10 == 0:
                        print(f'Counting NGrams for Markov Model: {(i / len(sequences) * 100):>5.2f}%', end='\r')
                    for n in range(0, order + 1):
                        for pos in range(len(seq) - n):
                            ngram = seq[pos:pos+n+1]
                            self.add_ngram_occurrence(ngram)

            duration = time.time() - start
            if verbose:
                print(f'Order {order} Markov Model of {len(sequences)} sequences built in {duration:.3f} seconds!')

    def __str__(self):
        return f'{self.order}-Order Markov NGram Object - Totals: {self.totals}'
    
    def __check_len(self, l):
        if l > self.order + 1 or l == 0:
            raise ValueError(f'Invalid NGram - Length of ngram must be between 1 and {self.order + 1} for an order {self.order} NGrams object (is {l})')

    @classmethod
    def __tuple_from_list(cls, l: list):
        t = (l[0],)
        for item in l[1:]:
            t += (item,)

        return t
    
    def __check_type(self, a):
        if not isinstance(a, self.__type):
            raise TypeError(f'Argument must be of type {self.__type} (is {type(a)})')

    def add_ngram_occurrence(self, ngram):
        self.__check_len(len(ngram))
        self.__check_type(ngram)
        self.ngram_freqs[len(ngram) - 1][MarkovModel.__tuple_from_list(ngram)] += 1
        self.totals[len(ngram) - 1] += 1

    def get_total_occurrences(self, ngram):
        self.__check_len(len(ngram))
        self.__check_type(ngram)
        return self.ngram_freqs[len(ngram) - 1][MarkovModel.__tuple_from_list(ngram)]
    
    def get_log_cond_prob(self, ngram):
        self.__check_len(len(ngram))
        self.__check_type(ngram)

        top = self.get_total_occurrences(ngram) + self.smoothing_weight
        bot = self.totals[0] if (len(ngram) == 1) else self.get_total_occurrences(ngram[:-1])
        bot += self.smoothing_weight * ((len(ngram) - 1) ** self.vocab_size)
        
        return math.log(top / bot) if top > 0 else sys.minint
    
    def get_log_sum_prob(self, seq: str):
        # TODO: how to handle empty sequences (high or low value?)
        # if (len(seq) == 0):
        #     raise ValueError(f'Invalid Sequence - Cannot get probability for an empty list')

        start = 0
        end = 1
        sum = 0

        while (end <= len(seq)):
            sum += self.get_log_cond_prob(seq[start:end])
            end += 1

            # ngrams can be at most order + 1 characters long:
            if (end - start > self.order + 1):
                start += 1
        return sum


In [4]:
texts_pos = (df_train.loc[df_train['target'] == 1])['text'].to_list()
texts_neg = (df_train.loc[df_train['target'] == 0])['text'].to_list()

MARKOV_ORDER = 10

model_pos = MarkovModel(MARKOV_ORDER, texts_pos, verbose=True)
model_neg = MarkovModel(MARKOV_ORDER, texts_neg, verbose=True)
model_pos2 = MarkovModel(MARKOV_ORDER, texts_pos, multithread=False, verbose=True)
model_neg2 = MarkovModel(MARKOV_ORDER, texts_neg, multithread=False, verbose=True)

def compare(sequence: str):
    pos1 = model_pos.get_log_sum_prob(sequence)
    pos2 = model_pos2.get_log_sum_prob(sequence)
    neg1 = model_neg.get_log_sum_prob(sequence)
    neg2 = model_neg2.get_log_sum_prob(sequence)

    print(f'Positives match: {pos1 == pos2}, Negatives match: {neg1 == neg2}')

compare("They should all die! All of them! Everything annihilated!")
compare("@sakuma_en If you pretend to feel a certain way the feeling can become genuine all by accident. -Hei (Darker than Black) #manga #anime")
compare("Shot 12 times. Found dead in cuffs after being involved in a car accident. Officers told ambulance not to treat him. https://t.co/MEUDJwaaNg")
compare("@NinaHoag - 'if you shred my Psych work our friendship would be annihilated")
compare("Why should a helicopter ambulance ride to transfer to a hospital 21 miles away cost $29800?")
compare("If you build an army of 100 lions and their leader is a dog in any fight the lions will die like a dog.")
compare("One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1424")

Order 10 Markov Model of 3271 sequences built in 0.567 seconds!
Order 10 Markov Model of 4342 sequences built in 0.667 seconds!
Order 10 Markov Model of 3271 sequences built in 2.957 seconds!
Order 10 Markov Model of 4342 sequences built in 3.560 seconds!
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True


In [5]:
class ConfusionMatrix:
    tp: int
    fp: int
    tn: int
    fn: int

    def __init__(self, tp: int = 0, tn: int = 0, fp: int = 0, fn: int = 0):
        self.tp = tp
        self.fp = fp
        self.tn = tn
        self.fn = fn

    def __str__(self):
        total = self.tp + self.fp + self.tn + self.fn
        return f'Total: {total:.1f}, TP: {self.tp:.1f}, TN: {self.tn:.1f}, FP: {self.fp:.1f}, FN: {self.fn:.1f}, Prc: {self.get_precision():.3f}, Rec. {self.get_recall():.3f}, F1: {self.get_f1():.3f}'
    
    def get_f1(self):
        return 2 * ((self.get_precision() * self.get_recall()) / (self.get_precision() + self.get_recall()))
    
    def get_precision(self):
        return self.tp / (self.tp + self.fp)

    def get_recall(self):
        return self.tp / (self.tp + self.fn) 

    def add(self, rhs: 'ConfusionMatrix'):
        self.tp += rhs.tp
        self.fp += rhs.fp
        self.tn += rhs.tn
        self.fn += rhs.fn

    def div(self, divisor):
        self.tp /= divisor
        self.fp /= divisor
        self.tn /= divisor
        self.fn /= divisor

    @classmethod
    def average(cls, matrices: list['ConfusionMatrix']):
        result = ConfusionMatrix()
        for cm in matrices:
            result.add(cm)
        result.div(len(matrices))

        return result
        

In [6]:
class MarkovClassifier:
    model_pos: MarkovModel
    model_neg: MarkovModel
    order: int

    def __init__(self, order: int, positives: list, negatives: list, verbose: bool = False, 
                 vocab_size: int = MARKOV_DEF_VOCAB_SIZE, smoothing_weight: float = MARKOV_DEF_SMOOTHING_WEIGHT):
        
        self.model_pos = MarkovModel(order, positives, multithread=True, verbose=verbose, vocab_size=vocab_size, smoothing_weight=smoothing_weight)
        self.model_neg = MarkovModel(order, negatives, multithread=True, verbose=verbose, vocab_size=vocab_size, smoothing_weight=smoothing_weight)
        self.order = order

    def predict(self, seq) -> bool:
        return self.model_pos.get_log_sum_prob(seq) > self.model_neg.get_log_sum_prob(seq)

    def eval(self, positives: list, negatives: list) -> ConfusionMatrix:

        res = ConfusionMatrix()

        for instance in positives:
            if self.predict(instance):
                res.tp += 1
            else:
                res.fn += 1

        for instance in negatives:
            if not self.predict(instance):
                res.tn += 1
            else:
                res.fp += 1
        
        return res
    

In [7]:
import regex as re

def get_crossval_sets(df: pd.DataFrame, num_sets: int):

    df_shuffled: pd.DataFrame = df.sample(frac=1)
    len_chunk = len(df) / num_sets

    sets = {}
    sets['train'] = [None] * num_sets
    sets['val'] = [None] * num_sets

    for i in range(num_sets):
        start = round(i * len_chunk)
        end = round((i + 1) * len_chunk)
        sets['val'][i] = df_shuffled.iloc[start:end,:]
        sets['train'][i] = df_shuffled[~df_shuffled.index.isin(sets['val'][i].index)]

    return sets

def get_wordlist(text: str) -> list[str]:
    cleaned = text.lower()
    cleaned = re.sub(r'https?:\/\/.*[\r\n]*|[^\w\s]', ' ', cleaned)
    return cleaned.split()

def get_instances(df: pd.DataFrame, target: int, mapping = lambda x: x):
    return list(map(mapping, (df.loc[df['target'] == target])['text'].to_list()))

In [17]:
def test_orders(df: pd.DataFrame, num_sets: int = 5, mapping = lambda x: x, order_min: int = 0, order_max: int = 15, 
                print_results: bool = False, vocab_size: int = MARKOV_DEF_VOCAB_SIZE, smoothing_weight: float = MARKOV_DEF_SMOOTHING_WEIGHT):

    order_max += 1
    sets = get_crossval_sets(df, num_sets)
    f1_scores = [0] * (order_max - order_min)

    for order in range(order_min, order_max):
        res = ConfusionMatrix()
        time_training = 0.0
        time_validation = 0.0

        for i in range(num_sets):
            start = time.time()
            classifier = MarkovClassifier(order, get_instances(sets['train'][i], 1, mapping), 
                                          get_instances(sets['train'][i], 0, mapping), 
                                          vocab_size=vocab_size, smoothing_weight=smoothing_weight)
            
            time_training += time.time() - start
            start = time.time()
            cm = classifier.eval(get_instances(sets['val'][i], 1, mapping), get_instances(sets['val'][i], 0, mapping))
            
            time_validation += time.time() - start
            res.add(cm)

        res.div(len(sets['train']))
        f1_scores[order - order_min] = res.get_f1()

        if print_results:
            print(f'Order {order} results: ', end='')
            print(res)
            print(f'Training: [Tot.: {time_training:5.3f}s, Avg.: {time_training / num_sets:5.3f}s], Validation: [Tot.: {time_validation:5.3f}s, Avg.: {time_validation / num_sets:5.3f}s]')

    max_f1_index = max((v, i) for i, v in enumerate(f1_scores))[1]
    return (max_f1_index, f1_scores[max_f1_index])


In [24]:
NUM_SETS = 5

print(f'Best order word based: {test_orders(df_train, NUM_SETS, get_wordlist, order_max=6, vocab_size=200, print_results=True)}')
print(f'Best order character based: {test_orders(df_train, NUM_SETS, print_results=True)}')

Order 0 results: Total: 1522.6, TP: 503.6, TN: 677.8, FP: 190.6, FN: 150.6, Prc: 0.725, Rec. 0.770, F1: 0.747
Training: [Tot.: 0.528s, Avg.: 0.106s], Validation: [Tot.: 0.219s, Avg.: 0.044s]
Order 1 results: Total: 1522.6, TP: 446.6, TN: 193.4, FP: 675.0, FN: 207.6, Prc: 0.398, Rec. 0.683, F1: 0.503
Training: [Tot.: 0.650s, Avg.: 0.130s], Validation: [Tot.: 0.329s, Avg.: 0.066s]
Order 2 results: Total: 1522.6, TP: 515.8, TN: 462.2, FP: 406.2, FN: 138.4, Prc: 0.559, Rec. 0.788, F1: 0.654
Training: [Tot.: 0.749s, Avg.: 0.150s], Validation: [Tot.: 0.392s, Avg.: 0.078s]
Order 3 results: Total: 1522.6, TP: 560.2, TN: 331.8, FP: 536.6, FN: 94.0, Prc: 0.511, Rec. 0.856, F1: 0.640
Training: [Tot.: 0.871s, Avg.: 0.174s], Validation: [Tot.: 0.437s, Avg.: 0.087s]
Order 4 results: Total: 1522.6, TP: 570.0, TN: 306.8, FP: 561.6, FN: 84.2, Prc: 0.504, Rec. 0.871, F1: 0.638
Training: [Tot.: 0.901s, Avg.: 0.180s], Validation: [Tot.: 0.465s, Avg.: 0.093s]
Order 5 results: Total: 1522.6, TP: 571.6, TN: 

In [39]:
df_test = pd.read_csv(PATH_DATA_TEST)

classifier = MarkovClassifier(0, get_instances(df_train, 1, get_wordlist), get_instances(df_train, 0, get_wordlist), verbose=True)
df_test['target'] = df_test['text'].apply(lambda seq: classifier.predict(get_wordlist(seq))).astype(int)

display(df_test)

Order 0 Markov Model of 3271 sequences built in 0.058 seconds!
Order 0 Markov Model of 4342 sequences built in 0.045 seconds!


,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1


1444


In [40]:
df_submission = df_test[['id', 'target']]
display(df_submission)
df_submission.to_csv(os.path.join(DIR_DATA, 'submission.csv'), index=False)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [14]:
array = [1, 4, 2, 20, 18, -4]

print(max((v, i) for i, v in enumerate(array))[1])

20
